In [19]:
#HERE MISSING VALUES ARE DEALING WITH DIFFERENT METHOD
#1 drop columns with missing value
#2 imputation

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [21]:
df = pd.read_csv('train.csv')
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [22]:
#set target variable
y = df['SalePrice']

In [23]:
#set feature
X_possible = df.drop(['SalePrice'], axis = 1)

In [6]:
#To make life simple revome columns having dtype=object (string)
X = X_possible.select_dtypes(exclude='object')
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


In [7]:
#Now X contains all the columns having numerical values
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, train_size=0.7, test_size=0.3)

In [8]:
#Drop columns with missing values (1st method)

In [9]:
miss_col = [col for col in X_train.columns if X_train[col].isnull().any()]
miss_col

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [10]:
#define training and test set without missing columns
X_drop_train = X_train.drop(miss_col, axis=1)
X_drop_val = X_val.drop(miss_col, axis=1)

In [11]:
# now define function that calculates the score based on mae(mean absolute error)
def score_mae(X_train, X_val, y_train, y_val):
    my_model = RandomForestRegressor(n_estimators=10,random_state=0)
    my_model.fit(X_train, y_train)
    prediction = my_model.predict(X_val)
    mae = mean_absolute_error(y_val, prediction)
    return mae 

In [12]:

print('MAE score from 1stMethod:\t', (score_mae(X_drop_train, X_drop_val, y_train, y_val)))

MAE score from 1stMethod:	 19458.681506849316


2nd Method imputation let us use SimpleImputer that replace null by mean value of each column 

In [14]:
from sklearn.impute import SimpleImputer

In [15]:
my_impute = SimpleImputer()

In [16]:
X_train_impute = pd.DataFrame(my_impute.fit_transform(X_train))
X_val_impute = pd.DataFrame(my_impute.fit_transform(X_val))

In [17]:
#Imputation removed column names so that need to put them back
X_train_impute.columns = X_train.columns
X_val_impute.columns = X_val.columns

In [18]:
print('MAE score from 2ndMethod:\t', score_mae(X_train_impute, X_val_impute, y_train, y_val))

MAE score from 2ndMethod:	 19203.844520547947
